# Simulate Filament Dynamics under Follower-force compression and extension
	- Simulates filament dynamics from different initial orientations.
	- Computes filament orientation before and after one activity cycle (compressive-extensional) cycle

In [5]:
import sys
if 'init_modules' in globals().keys():
    # second or subsequent run: remove all but initially loaded modules
    for m in sys.modules.keys():
        if m not in init_modules:
            del(sys.modules[m])
else:
    # first run: find out which modules were initially loaded
    init_modules = sys.modules.keys()

from pyfilaments.activeFilaments import activeFilament
import numpy as np
from scipy import signal
from scipy import interpolate
import matplotlib.pyplot as plt
import os
from sys import platform

from joblib import Parallel, delayed
import multiprocessing

# Check which platform
if platform == "linux" or platform == "linux2":
    print("linux system")
    # root_path = '/home/deepak/LacryModelling_Local/SimulationData'
    root_path = '/home/deepak/Dropbox/LacryModeling/ModellingResults'


elif platform == 'darwin':
    print("OSX system")
    root_path = '/Users/deepak/Dropbox/LacryModeling/'

linux system


## Setup filament parameters

In [6]:
radius = 1
bc = {0:'clamped', -1:'free'} # Filament BC

# Bond length
b0 = 2.1

# Activity profile parameters


n_activity_cycles = 1.5 	# No:of activity cycles we want to simulate
activity_timescale = 750	
duty_cycle = 0.5

Tf = n_activity_cycles*activity_timescale
time_step_save = 1
Npts = int(Tf/time_step_save)

init_angle_array = np.linspace(0, 3*np.pi/8, 50)

# bending_stiffness_array = [90, 95, 100]
k = 25

activity_strength_array = np.linspace(0.5, 3, 20)
# activity_strength_array = [1.07692307692308, 1.14102564102564, 2.61538461538461]

## Batch simulation function

In [7]:
root_path = '/home/deepak/ActiveFilamentsSearch_backup_3/FollowerForce_BucklingDynamics_Simulations'

def run_parametric_simulation(pid, init_angle, activity_strength, activity_timescale, Tf, save_path):


    fil = activeFilament(dim = 3, Np = 32, radius = radius, b0 = b0*radius, k = k, S0 = 0, D0 = activity_strength, bc = bc)

    print(init_angle)
    
    # Phase angle is Pi since we start the simulation with an extension followed by compression + extension.
    fil.simulate(Tf, Npts, save = True, overwrite = False, path = save_path, sim_type = 'point', 
    init_condition = {'shape':'line', 'angle': init_angle}, 
    activity={'type':'square-wave','activity_timescale':activity_timescale, 'duty_cycle':duty_cycle, 'start phase':np.pi},
                note = str(round(init_angle, 2)))


## Run a parallel batch simulation for different parameters and initial angles

In [16]:
num_cores = multiprocessing.cpu_count()

num_cores = 12


for activity_strength in activity_strength_array:

    results = Parallel(n_jobs = num_cores,  verbose=10)(delayed(run_parametric_simulation)(pid, init_angle, activity_strength, activity_timescale, Tf, root_path) for pid, init_angle in enumerate(init_angle_array))


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    5.4s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    9.1s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   13.1s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   17.0s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   22.1s remaining:   11.4s
[Parallel(n_jobs=12)]: Done  39 out of  50 | elapsed:   23.8s remaining:    6.7s
[Parallel(n_jobs=12)]: Done  45 out of  50 | elapsed:   26.5s remaining:    2.9s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:   27.7s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    4.0s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    7.6s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   12.1s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   14.8s
[Parallel(n_jobs=12)]: Done  33 out 

[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:   24.8s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    3.9s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    7.2s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   11.7s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   14.4s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   19.5s remaining:   10.0s
[Parallel(n_jobs=12)]: Done  39 out of  50 | elapsed:   21.3s remaining:    6.0s
[Parallel(n_jobs=12)]: Done  45 out of  50 | elapsed:   23.3s remaining:    2.6s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:   24.8s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    3.8s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    7.1s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   11.6s
[Parallel(n_jobs=12)]: Done

[Parallel(n_jobs=12)]: Done  45 out of  50 | elapsed:   23.1s remaining:    2.6s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:   25.2s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    3.8s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    7.1s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   11.7s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   14.7s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   19.9s remaining:   10.2s
[Parallel(n_jobs=12)]: Done  39 out of  50 | elapsed:   21.7s remaining:    6.1s
[Parallel(n_jobs=12)]: Done  45 out of  50 | elapsed:   23.8s remaining:    2.6s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:   25.3s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    3.9s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    7.1s
[Paralle

[Parallel(n_jobs=12)]: Done  39 out of  50 | elapsed:   22.8s remaining:    6.4s
[Parallel(n_jobs=12)]: Done  45 out of  50 | elapsed:   24.4s remaining:    2.7s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:   26.5s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    4.0s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    7.6s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   12.1s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   15.3s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   20.5s remaining:   10.6s
[Parallel(n_jobs=12)]: Done  39 out of  50 | elapsed:   23.1s remaining:    6.5s
[Parallel(n_jobs=12)]: Done  45 out of  50 | elapsed:   24.6s remaining:    2.7s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:   26.8s finished


## Batch simulations for different activity time-scales

In [9]:
# Store each activity timescale set of simulations in a specific folder.

num_cores = multiprocessing.cpu_count()

num_cores = 12

activity_timescale_array = [400, 450, 550, 650, 750, 850, 950]

activity_strength = 1.65

for activity_timescale in activity_timescale_array:
    Tf = n_activity_cycles*activity_timescale
    time_step_save = 1
    Npts = int(Tf/time_step_save)
    
    # Create a new folder for each activity timescale.
    save_path = os.path.join(root_path, 'activity_timescale_{}'.format(int(activity_timescale)))
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    print('Running Activity time-scale: {}...'.format(activity_timescale))
    
    for activity_strength in activity_strength_array:
    
        results = Parallel(n_jobs = num_cores,  verbose=10)(delayed(run_parametric_simulation)(pid, init_angle, activity_strength, activity_timescale, Tf, save_path) for pid, init_angle in enumerate(init_angle_array))


Running Activity time-scale: 400...


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    3.4s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    5.4s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    7.9s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    9.6s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   12.2s remaining:    6.3s
[Parallel(n_jobs=12)]: Done  39 out of  50 | elapsed:   13.2s remaining:    3.7s
[Parallel(n_jobs=12)]: Done  45 out of  50 | elapsed:   14.4s remaining:    1.6s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:   15.3s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    2.1s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    4.1s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    6.4s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    8.1s
[Parallel(n_jobs=12)]: Done  33 out 

[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:   13.5s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    2.1s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    3.8s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    6.3s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    7.8s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   10.6s remaining:    5.4s
[Parallel(n_jobs=12)]: Done  39 out of  50 | elapsed:   11.4s remaining:    3.2s
[Parallel(n_jobs=12)]: Done  45 out of  50 | elapsed:   12.6s remaining:    1.4s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:   13.4s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    2.0s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    3.9s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    6.2s
[Parallel(n_jobs=12)]: Done

Running Activity time-scale: 450...


[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    2.5s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    5.0s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    7.4s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    9.6s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   12.5s remaining:    6.4s
[Parallel(n_jobs=12)]: Done  39 out of  50 | elapsed:   13.8s remaining:    3.9s
[Parallel(n_jobs=12)]: Done  45 out of  50 | elapsed:   14.9s remaining:    1.7s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:   15.9s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    2.4s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    4.6s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    7.2s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    9.1s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   11.7s remaining:    6.0s
[Parallel(n_jobs=12)]: Done  39 

...

[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    4.7s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    9.1s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   14.0s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   18.2s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   23.3s remaining:   12.0s
[Parallel(n_jobs=12)]: Done  39 out of  50 | elapsed:   25.9s remaining:    7.3s
[Parallel(n_jobs=12)]: Done  45 out of  50 | elapsed:   27.9s remaining:    3.1s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:   30.5s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    4.5s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    8.6s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   13.3s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   17.2s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   22.3s remaining:   11.5s
[Parallel(n_jobs=12)]: Done  39 

[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    4.5s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    8.3s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   14.0s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   17.1s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   23.1s remaining:   11.9s
[Parallel(n_jobs=12)]: Done  39 out of  50 | elapsed:   24.9s remaining:    7.0s
[Parallel(n_jobs=12)]: Done  45 out of  50 | elapsed:   27.6s remaining:    3.1s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:   29.2s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    4.6s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    8.4s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   14.0s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   17.1s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   23.3s remaining:   12.0s
[Parallel(n_jobs=12)]: Done  39 

Running Activity time-scale: 950...


[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    5.3s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    9.9s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   15.8s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   19.7s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   26.2s remaining:   13.5s
[Parallel(n_jobs=12)]: Done  39 out of  50 | elapsed:   28.8s remaining:    8.1s
[Parallel(n_jobs=12)]: Done  45 out of  50 | elapsed:   31.5s remaining:    3.5s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:   33.8s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    5.1s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    9.5s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   15.2s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   18.7s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   25.4s remaining:   13.1s
[Parallel(n_jobs=12)]: Done  39 

[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    5.2s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    9.5s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   15.7s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   19.1s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   26.3s remaining:   13.6s
[Parallel(n_jobs=12)]: Done  39 out of  50 | elapsed:   28.1s remaining:    7.9s
[Parallel(n_jobs=12)]: Done  45 out of  50 | elapsed:   31.6s remaining:    3.5s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:   33.0s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    5.1s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    9.3s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   15.2s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   18.4s
[Parallel(n_jobs=12)]: Done  33 out of  50 | elapsed:   25.2s remaining:   13.0s
[Parallel(n_jobs=12)]: Done  39 